##Обучение пайплайна

- Загрузим данные https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction
- Соберем пайплайн с простейшим препроцессингом (tfidf) на текстовых данных
- Обучим логистическую регрессию и сохраним на диск предобученный пайплайн

In [ ]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from sklearn.metrics import f1_score
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score
#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

In [ ]:
df = pd.read_csv("./fake_job_postings.csv")
df.head(3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['fraudulent'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [ ]:
features = ['description', 'company_profile', 'benefits']
target = 'fraudulent'

In [ ]:
#combine
description = Pipeline([
                ('imputer', TextImputer('description', '')),
                ('selector', ColumnSelector(key='description')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])
company_profile = Pipeline([
                ('imputer', TextImputer('company_profile', '')),
                ('selector', ColumnSelector(key='company_profile')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])
benefits = Pipeline([
                ('imputer', TextImputer('benefits', '')),
                ('selector', ColumnSelector(key='benefits')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])

feats = FeatureUnion([('description', description),
                      ('company_profile', company_profile),
                      ('benefits', benefits)])

In [ ]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

In [ ]:
pipeline.steps

In [ ]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)